In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import sys
sys.path.append("../")
from component.message import cm

In [ ]:
from component.model.recipe import Recipe
from component.tile.custom_aoi_tile import AoiTile
from component.tile.dashboard_tile import DashboardTile
from component.widget.alert_state import AlertState

from pathlib import Path
import json
from component.scripts.plots import get_individual_charts
import ipyvuetify as v

In [ ]:
build_alert = AlertState()
build_alert

In [ ]:
build_alert

In [ ]:
recipe = Recipe()

In [ ]:
# We need the AoiTile so it can create the SeplanAoi and will 
# set the feature collection in the model
aoi_tile = AoiTile(recipe=recipe)

In [ ]:
recipe.load("/home/dguerrero/module_results/se.plan/recipes/Myanmar_example.json")

In [ ]:
dashboard_tile = DashboardTile(recipe=recipe)

In [ ]:
dashboard_tile.reset()
dashboard_tile.summary_stats = json.loads(Path("data/summary_stats.json").read_text())["results"]

In [ ]:
suitability_data = {area_name:data["suitability"] for area_name, data in dashboard_tile.summary_stats.items()}
# suitability_data

In [ ]:
dashboard_tile

# Test suitability chart

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append("../")

In [5]:
from component.scripts.plots import get_stacked_bars_chart, get_bars_chart, parse_layer_data, get_suitability_charts
import json
from component.widget.suitability_table import get_summary_table
from pathlib import Path
import ipyvuetify as v

In [6]:
summary_stats = json.loads(Path("data/summary_stats_double.json").read_text())["results"]

In [7]:
summary_stats2 = json.loads(Path("data/summary_stats.json").read_text())["results"]

In [8]:
summary_stats3 = json.loads(Path("data/summary_stats_single.json").read_text())["results"]

In [9]:
from component.parameter.vis_params import gradient

In [10]:
palettes = {"green": ["#EBFAF2", "#66c2a4", "#006d2c"]}
# palettes = {"green": ["#98FB98", "#32CD32", "#006400"]}

no_data_color = ["#353535"]  # the color used for values filtered by constraints
gradient_palette = gradient(levels=5, palette=palettes["green"])
gradient_palette

['#ebfaf2', '#a8decb', '#66c2a4', '#339768', '#006d2c']

In [12]:
chart = get_suitability_charts([summary_stats, summary_stats2,summary_stats3], test=True)
chart

Card(children=[CardText(children=[EChartsWidget(height='200', option=Option(angleAxis=None, aria=None, axisPoi…

In [13]:
suitability_table = get_summary_table([summary_stats, summary_stats2, summary_stats3], "both")
suitability_table

SimpleTable(children=[Html(children=[Html(children=[Html(children=['Area name', IconBtn(children=[Icon(childre…

In [20]:
v.theme.dark=not v.theme.dark

In [ ]:
chart.theme='light'
chart.renderer='svg'

In [ ]:
chart.theme='dark'
chart.renderer='svg'

In [ ]:
layer_id = "plantation_growth_rates"
layer_data = recipe.seplan.benefit_model.get_layer_data("plantation_growth_rates")
aoi_names, values, colors = parse_layer_data(summary_results, layer_id)

In [23]:
aoi_names, values, colors, series_names = [], [], [], []
for layer_id in recipe.seplan.cost_model.ids:

    layer_data = recipe.seplan.cost_model.get_layer_data(layer_id)

    aoi_name, value, color = parse_layer_data(summary_results, layer_id)
    aoi_names.append(aoi_name)
    values.append(value)
    colors.append(color)
    series_names.append(layer_data["name"])

chart = get_bars_chart(
    aoi_names[0], values, series_names, series_colors = ['#4CAF50', '#2196F3'
    ], custom_item_color=False, custom_item_colors=colors, bars_width=70
)
chart

NameError: name 'recipe' is not defined

In [ ]:
print(chart)

In [ ]:
chart

In [21]:
from component.widget.dashboard_layer_panels import LayerFull

In [22]:
layer_data = recipe.seplan.cost_model.get_layer_data(layer_id)

NameError: name 'recipe' is not defined

In [ ]:
LayerFull(layer_data, chart)

### Test stats 

In [39]:
ee_features = recipe.seplan.aoi_model.get_ee_features()
ee_features

{'COL_Antioquia': {'ee_feature': <pygaul.AdmItems at 0x76116e9f0bb0>,
  'color': '#76591e'}}

In [40]:
from component.tile.dashboard_tile import DashboardTile
from component.widget.alert_state import AlertState

In [41]:
alert = AlertState()
alert

AlertState(class_='d-none', layout=None, old_class='mt-5', style_='margin: 0 !important;', text=True, type='in…

In [68]:
dashboard_tile = DashboardTile(recipe)

In [69]:
dashboard_tile

DashboardTile(children=[DashToolbar(children=[Tooltip(children=['Save dashboard statistics as .csv'], close_de…

## Test statistics from recipe

In [24]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [26]:
import sys
sys.path.append("../")
from component.scripts.statistics import get_summary_statistics
from component.model.recipe import Recipe
from component.scripts.seplan import reduce_constraints
import ee
ee.Initialize()

In [28]:
# Load a previously created recipe
recipe = Recipe()
recipe.load("/home/dguerrero/module_results/se.plan/recipes/antioquia_1.json")
recipe.seplan_aoi.aoi_model.set_object(); # we have to do this manually
aoi = recipe.seplan_aoi.aoi_model.feature_collection

In [31]:
summary_stats = get_summary_statistics(recipe)
summary_stats

{'': {'COL_Antioquia': {'benefit': [{'biodiversity_intactness': {'total': [0.9141905697146238],
      'values': [0.9038905752014645]}},
    {'endangered_species': {'total': [14.587265048855699],
      'values': [14.337335770802504]}},
    {'ground_carbon': {'total': [85.2069566719725],
      'values': [103.47732748252137]}},
    {'forest_job': {'total': [0.003911315734834424],
      'values': [0.0039113159113282284]}},
    {'woodfuel_harvest': {'total': [0.2248397385459174],
      'values': [0.2248397464960843]}},
    {'plantation_growth_rates': {'total': [37.86204668873121],
      'values': [37.73709455798074]}}],
   'color': '#76591e',
   'constraint': [{'treecover_with_potential': {'total': [6315210.808061779],
      'values': [44.246559142712414]}},
    {'city_access': {'total': [6315210.808062925],
      'values': [0.0014091790618350813]}}],
   'cost': [{'opportunity_cost': {'total': [9195.687180859843],
      'values': [6848.380759528549]}},
    {'implementation_cost': {'total': 

## Calculate the image percent cover pixel area

In [33]:
from component.scripts.statistics import get_image_percent_cover_pixelarea
from component.scripts.seplan import mask_image

In [34]:
aoi = recipe.seplan.aoi_model.feature_collection

In [35]:
asset_id = recipe.seplan.constraint_model.assets[0]
data_type = recipe.seplan.constraint_model.data_type[0]
maskout_values = recipe.seplan.constraint_model.values[0]

asset_id, data_type, maskout_values

('projects/se-plan/assets/inputLayers/total_rest_pot_gt_treecoverfrac_masks',
 'binary',
 [0])

In [36]:
# maskout any image from the model

masked_image = mask_image(asset_id, data_type, maskout_values)

In [37]:
get_image_percent_cover_pixelarea(masked_image, aoi, "perro").getInfo()

{'perro': {'total': [6315210.808061779], 'values': [44.246559142712414]}}

In [38]:
from sepal_ui.mapping import SepalMap
import ee

ee.Initialize()

In [ ]:
map_ = SepalMap()
map_.addLayer(masked_image.unmask(0).randomVisualizer().clip(aoi))
map_

## Calculate the image sum on COSTS

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from component.scripts.statistics import get_image_sum
from component.scripts.seplan import reduce_constraints, asset_to_image

In [ ]:
aoi = recipe.seplan.aoi_model.feature_collection

In [ ]:
asset_id = recipe.seplan.cost_model.assets[0]
constraint_list = recipe.seplan.get_masked_constraints_list()
mask_out_areas = reduce_constraints(constraint_list)

In [ ]:
image = asset_to_image(asset_id)

In [ ]:
sum_img = image.updateMask(mask_out_areas).reduceRegion(
    reducer=ee.Reducer.sum(),
    geometry=aoi,
    scale=100,
    maxPixels=1e13,
)
sum_img.values().get(0).getInfo()

In [ ]:
get_image_sum(image, aoi, mask_out_areas, "name").getInfo()

In [ ]:
area_ha = (
    ee.Image.pixelArea()
    .divide(10000)
    .reduceRegion(
        reducer=ee.Reducer.sum(),
        geometry=aoi,
        scale=100,
        maxPixels=1e13,
    )
    .getInfo()
)

In [ ]:
map_.addLayer(constraint_list[0][0].clip(aoi), {}, "mask")

In [ ]:
map_ = SepalMap()
map_.addLayer(mask_out_areas.randomVisualizer().clip(aoi))
map_